# GPT-J-6B Parallel Architecture Test

**Paper #3 - Pythia Anomaly Investigation**

**Date:** 2026-01-05

**Purpose:** Validate the PARALLEL ATTENTION ARCHITECTURE hypothesis

---

## Background

GPT-2 validation REJECTED the "LayerNorm = Dampening" hypothesis:
- Pythia-6.9B (LayerNorm): Gain = 0.80 (DAMPENING)
- GPT-2 (LayerNorm): Gain = 1.01-1.16 (EXPANSION)

**New Hypothesis:** It's not LayerNorm that causes dampening - it's the **Parallel Attention Architecture**.

---

## Architectural Difference

| Model | Architecture | Attention | Expected Gain |
|-------|-------------|-----------|---------------|
| GPT-2 | Sequential | h = x + Attn(x); out = h + MLP(h) | > 1.0 ✓ |
| Pythia | GPT-NeoX (Parallel) | out = x + Attn(x) + MLP(x) | < 1.0 ✓ |
| **GPT-J** | **GPT-NeoX (Parallel)** | **out = x + Attn(x) + MLP(x)** | **< 1.0 ?** |

---

## Hypothesis

**If GPT-J-6B shows Gain < 1.0, this CONFIRMS that Parallel Attention causes dampening.**

**If GPT-J-6B shows Gain > 1.0, the hypothesis is REJECTED and we need to investigate further.**

In [ ]:
# Cell 1: Setup
!pip install -q transformers accelerate scipy seaborn

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTJForCausalLM
from scipy.stats import entropy, spearmanr, pearsonr, ttest_1samp
import gc
import json
from datetime import datetime
from google.colab import files
import os
import warnings
warnings.filterwarnings('ignore')

# HF Token handling - GPT-J may need token
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("HF_TOKEN loaded from Colab secrets")
except:
    HF_TOKEN = None
    print("No HF_TOKEN found")

# Configure visualization
plt.style.use('seaborn-v0_8-paper')
sns.set_context("talk")

# Global timestamp for all files
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"\nSession timestamp: {TIMESTAMP}")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Cell 2: PROMPT SET (Same as Grand Unified Benchmark)

PROMPT_DATASET = {
    "Factual": [
        "The capital city of France is",
        "The atomic number of oxygen is",
        "Water boils at a temperature of",
        "The largest planet in our solar system is",
        "The currency used in Japan is"
    ],
    "Syntactic": [
        "The agreement, which, notwithstanding the fact that it was signed only yesterday, effectively binds all parties immediately, stipulates that",
        "Although the weather was extremely cold, and despite the fact that they had no coats, the children decided to",
        "The professor, having reviewed the complex derivation multiple times without finding the error, finally realized that",
        "To imply that such a fundamental shift in policy could occur without significant public debate is to suggest that",
        "Not only did the experiment fail to yield the expected results, but it also demonstrated that the initial hypothesis was"
    ],
    "Cliche": [
        "The true meaning of happiness is often found in",
        "Actions speak louder than",
        "It is what it is, and we must simply",
        "Time heals all",
        "Life is a journey, not a"
    ],
    "Novel": [
        "The epistemological implications of quantum decoherence suggest that the observer is",
        "If consciousness creates reality, then the paradox of the unobserved electron implies",
        "The intersection of baroque architecture and cybernetic theory creates a space where",
        "Calculating the trajectory of a hyperspace jump requires factoring in the variability of",
        "The symbiotic relationship between fungal mycelium and digital neural networks results in"
    ],
    "Nonsense": [
        "Table sky run blue jump quickly under over",
        "Purple idea furiously sleep colorless green",
        "Clock river dance potato seven fast",
        "Window eat loud tomorrow yellow under",
        "Fish bicycle logic cloud mountain swim"
    ]
}

CATEGORY_METADATA = {
    "Factual": {"expected_entropy": "medium", "complexity": 1},
    "Syntactic": {"expected_entropy": "medium", "complexity": 5},
    "Cliche": {"expected_entropy": "low", "complexity": 2},
    "Novel": {"expected_entropy": "high", "complexity": 4},
    "Nonsense": {"expected_entropy": "very_high", "complexity": 3}
}

total_prompts = sum(len(v) for v in PROMPT_DATASET.values())
print(f"Total prompts: {total_prompts} (5 categories x 5 prompts)")
for cat, prompts in PROMPT_DATASET.items():
    print(f"  {cat}: {len(prompts)} prompts")

In [ ]:
# Cell 3: MODEL CONFIGURATION

# GPT-J-6B - The Test Subject
MODEL_TO_TEST = {
    "GPT-J-6B": {
        "hf_path": "EleutherAI/gpt-j-6b",
        "architecture": "GPT-NeoX (Parallel)",
        "norm_type": "LayerNorm",
        "params": "6B",
        "expected_gain": "< 1.0 (if hypothesis correct)",
        "role": "Parallel Architecture Validation"
    }
}

# Reference values from prior experiments
REFERENCE_MODELS = {
    # Parallel Attention (GPT-NeoX)
    "Pythia-6.9B": {"gain": 0.80, "architecture": "GPT-NeoX (Parallel)", "norm": "LayerNorm"},
    
    # Sequential Attention (GPT-2)
    "GPT2-XL": {"gain": 1.02, "architecture": "Sequential", "norm": "LayerNorm"},
    "GPT2-Large": {"gain": 1.01, "architecture": "Sequential", "norm": "LayerNorm"},
    "GPT2-Medium": {"gain": 1.16, "architecture": "Sequential", "norm": "LayerNorm"},
    
    # RMSNorm models (various architectures)
    "Mistral-7B": {"gain": 1.11, "architecture": "LLaMA-like", "norm": "RMSNorm"},
    "LLaMA-3.1-8B": {"gain": 1.48, "architecture": "LLaMA", "norm": "RMSNorm"},
    "Gemma-7B": {"gain": 2.31, "architecture": "Gemma", "norm": "RMSNorm"}
}

print("="*70)
print("PARALLEL ARCHITECTURE HYPOTHESIS TEST")
print("="*70)
print("\nTest Subject:")
for name, info in MODEL_TO_TEST.items():
    print(f"  {name}")
    print(f"    Path: {info['hf_path']}")
    print(f"    Architecture: {info['architecture']}")
    print(f"    Norm: {info['norm_type']}")
    print(f"    Expected: {info['expected_gain']}")

print("\nReference Models (Parallel Architecture):")
for name, info in REFERENCE_MODELS.items():
    if "Parallel" in info["architecture"]:
        print(f"  {name}: Gain={info['gain']:.2f} ({info['architecture']})")

print("\nReference Models (Sequential Architecture):")
for name, info in REFERENCE_MODELS.items():
    if "Sequential" in info["architecture"]:
        print(f"  {name}: Gain={info['gain']:.2f} ({info['norm']})")

In [ ]:
# Cell 4: MEASUREMENT ENGINE (GPT-J Compatible)

def get_layer_list(model):
    """Get the transformer layers from different model architectures."""
    # GPT-J style: model.transformer.h
    if hasattr(model, 'transformer') and hasattr(model.transformer, 'h'):
        return model.transformer.h
    # GPT-2 style: model.transformer.h
    elif hasattr(model, 'transformer') and hasattr(model.transformer, 'h'):
        return model.transformer.h
    # LLaMA/Mistral/Gemma style: model.model.layers
    elif hasattr(model, 'model') and hasattr(model.model, 'layers'):
        return model.model.layers
    # Pythia style: model.gpt_neox.layers
    elif hasattr(model, 'gpt_neox') and hasattr(model.gpt_neox, 'layers'):
        return model.gpt_neox.layers
    else:
        raise ValueError(f"Unknown model architecture: {type(model)}")

def measure_thermodynamics(model, tokenizer, text, device='cuda'):
    """Measure residual stream gain and output entropy for a given input."""
    
    # Tokenize - handle padding token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Hooks to capture residual stream norms
    norms = []
    
    def get_norm_hook():
        def hook(module, input, output):
            # Handle different output formats
            if isinstance(output, tuple):
                hidden_state = output[0]
            else:
                hidden_state = output
            
            # Ensure we're working with a tensor
            if hasattr(hidden_state, 'last_hidden_state'):
                hidden_state = hidden_state.last_hidden_state
            
            # Norm of last token - convert to float explicitly
            try:
                last_token_norm = float(torch.norm(hidden_state[0, -1]).cpu().item())
                norms.append(last_token_norm)
            except Exception as e:
                print(f"Warning: Could not compute norm: {e}")
        return hook
    
    # Register hooks on all layers
    handles = []
    try:
        layers = get_layer_list(model)
        for layer in layers:
            handles.append(layer.register_forward_hook(get_norm_hook()))
    except Exception as e:
        print(f"Warning: Could not register hooks: {e}")
        return None
    
    # Forward pass
    try:
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
    except Exception as e:
        print(f"Error in forward pass: {e}")
        for h in handles:
            h.remove()
        return None
    
    # Cleanup hooks
    for h in handles:
        h.remove()
    
    # Calculate metrics - explicit float conversion
    if len(norms) >= 2:
        last_gain = float(norms[-1] / norms[-2]) if norms[-2] > 0 else 1.0
        total_amp = float(norms[-1] / norms[0]) if norms[0] > 0 else 1.0
    else:
        last_gain = 1.0
        total_amp = 1.0
    
    # Output Entropy (next token distribution)
    last_token_logits = logits[0, -1, :].float()  # Ensure float32
    probs = torch.softmax(last_token_logits, dim=0).cpu().numpy().astype(np.float64)
    
    # Clip to avoid log(0) issues
    probs = np.clip(probs, 1e-10, 1.0)
    probs = probs / probs.sum()  # Renormalize
    
    ent = float(entropy(probs))
    
    # Top token and probability
    top_idx = int(torch.argmax(last_token_logits).item())
    top_prob = float(probs[top_idx])
    top_token = tokenizer.decode([top_idx])
    
    return {
        "last_gain": float(last_gain),
        "total_amp": float(total_amp),
        "entropy": float(ent),
        "top_token": str(top_token),
        "top_prob": float(top_prob),
        "n_layers": int(len(norms)),
        "all_norms": [float(n) for n in norms]
    }

print("Measurement engine ready (GPT-J compatible).")

In [ ]:
# Cell 5: LOAD GPT-J-6B AND RUN MEASUREMENTS

all_results = []

print("="*70)
print("LOADING GPT-J-6B (6 Billion Parameters)")
print("This may take a few minutes...")
print("="*70 + "\n")

model_name = "GPT-J-6B"
model_info = MODEL_TO_TEST[model_name]
hf_path = model_info["hf_path"]

print(f"Loading {model_name} ({model_info['params']})...")
print(f"  Path: {hf_path}")
print(f"  Architecture: {model_info['architecture']}")
print(f"  Norm: {model_info['norm_type']}")

try:
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        hf_path,
        token=HF_TOKEN if HF_TOKEN else None
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Load model with bfloat16 for memory efficiency
    model = AutoModelForCausalLM.from_pretrained(
        hf_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        token=HF_TOKEN if HF_TOKEN else None,
        low_cpu_mem_usage=True
    )
    model.eval()
    
    print(f"\nModel loaded successfully!")
    print(f"Number of layers: {len(get_layer_list(model))}")
    
    print(f"\nTesting {len(PROMPT_DATASET)} categories ({total_prompts} prompts)...\n")
    
    for category, prompts in PROMPT_DATASET.items():
        print(f"  {category}: ", end="")
        for i, prompt in enumerate(prompts):
            res = measure_thermodynamics(model, tokenizer, prompt)
            
            if res is not None:
                all_results.append({
                    "Model": str(model_name),
                    "Architecture": str(model_info["architecture"]),
                    "Norm_Type": str(model_info["norm_type"]),
                    "Params": str(model_info["params"]),
                    "Role": str(model_info["role"]),
                    "Category": str(category),
                    "Complexity": int(CATEGORY_METADATA[category]["complexity"]),
                    "Prompt": str(prompt),
                    "Prompt_Short": str(prompt[:40] + "..."),
                    "Entropy": float(res["entropy"]),
                    "Last_Gain": float(res["last_gain"]),
                    "Total_Amp": float(res["total_amp"]),
                    "Top_Token": str(res["top_token"]),
                    "Top_Prob": float(res["top_prob"]),
                    "N_Layers": int(res["n_layers"])
                })
                print(".", end="", flush=True)
            else:
                print("X", end="", flush=True)
        print(f" Done (n={len(prompts)})")
    
    # Cleanup
    del model
    del tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    print(f"\n{model_name} complete. Memory cleared.")
    
except Exception as e:
    print(f"\nFAILED: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "="*70)
print("MEASUREMENT COMPLETE")
print(f"Total measurements: {len(all_results)}")
print("="*70)

In [ ]:
# Cell 6: CREATE DATAFRAME & ANALYZE RESULTS

df = pd.DataFrame(all_results)

# Define filenames with global timestamp
CSV_FILE = f"gptj_parallel_test_{TIMESTAMP}.csv"
JSON_FILE = f"gptj_parallel_test_{TIMESTAMP}.json"
PNG_MAIN = f"gptj_parallel_test_{TIMESTAMP}.png"
PNG_ARCH = f"gptj_architecture_comparison_{TIMESTAMP}.png"

# Save raw results
df.to_csv(CSV_FILE, index=False)
print(f"Saved: {CSV_FILE}")

# Display summary
print("\n" + "="*70)
print("GPT-J-6B RESULTS SUMMARY")
print("="*70)

mean_gain = float(df['Last_Gain'].mean())
std_gain = float(df['Last_Gain'].std())
min_gain = float(df['Last_Gain'].min())
max_gain = float(df['Last_Gain'].max())

print(f"\nGPT-J-6B Last Layer Gain:")
print(f"  Mean:  {mean_gain:.4f}")
print(f"  Std:   {std_gain:.4f}")
print(f"  Range: [{min_gain:.4f}, {max_gain:.4f}]")

# Critical test
print("\n" + "="*70)
print("CRITICAL HYPOTHESIS TEST")
print("="*70)

print(f"\nHypothesis: Parallel Attention Architecture causes dampening (Gain < 1.0)")
print(f"\nReference:")
print(f"  Pythia-6.9B (Parallel): 0.80")
print(f"  GPT-2-XL (Sequential):  1.02")

print(f"\nResult:")
print(f"  GPT-J-6B (Parallel):    {mean_gain:.4f}")

if mean_gain < 1.0:
    print(f"\n  ✅ GPT-J-6B shows DAMPENING (Gain < 1.0)")
    print(f"  ✅ PARALLEL ARCHITECTURE HYPOTHESIS: CONFIRMED")
    hypothesis_status = "CONFIRMED"
else:
    print(f"\n  ❌ GPT-J-6B shows EXPANSION (Gain > 1.0)")
    print(f"  ❌ PARALLEL ARCHITECTURE HYPOTHESIS: REJECTED")
    hypothesis_status = "REJECTED"

print("\nMean Gain per Category:")
print(df.groupby('Category')['Last_Gain'].agg(['mean', 'std']).round(4))

In [ ]:
# Cell 7: STATISTICAL VALIDATION

print("\n" + "="*70)
print("STATISTICAL VALIDATION")
print("="*70)

# T-test against 1.0 (null hypothesis: gain = 1.0)
t_stat, p_val_two = ttest_1samp(df['Last_Gain'], 1.0)

# One-sided p-value
if t_stat < 0:
    p_one_sided = float(p_val_two / 2)  # Testing for < 1.0
    direction = "dampening"
else:
    p_one_sided = float(1 - p_val_two / 2)  # Testing for > 1.0
    direction = "expansion"

print(f"\nT-test: Is GPT-J-6B mean gain significantly different from 1.0?")
print(f"  t-statistic: {t_stat:.4f}")
print(f"  p-value (two-sided): {p_val_two:.6f}")
print(f"  p-value (one-sided, {direction}): {p_one_sided:.6f}")

if p_one_sided < 0.001:
    sig = "*** (p < 0.001)"
elif p_one_sided < 0.01:
    sig = "** (p < 0.01)"
elif p_one_sided < 0.05:
    sig = "* (p < 0.05)"
else:
    sig = "(not significant)"

print(f"  Significance: {sig}")

# Compare with Pythia
pythia_gain = 0.80
gpt2_gain = 1.02  # GPT2-XL

print(f"\nComparison:")
print(f"  GPT-J-6B:     {mean_gain:.4f} (This Test)")
print(f"  Pythia-6.9B:  {pythia_gain:.4f} (Reference - Same Architecture)")
print(f"  Difference:   {abs(mean_gain - pythia_gain):.4f}")

if abs(mean_gain - pythia_gain) < 0.15:
    print(f"\n  --> CONSISTENT: GPT-J and Pythia both show dampening")
elif mean_gain < 1.0 and pythia_gain < 1.0:
    print(f"\n  --> DIRECTIONALLY CONSISTENT: Both < 1.0 (dampening)")
else:
    print(f"\n  --> DIVERGENT: Different behavior despite same architecture")

In [ ]:
# Cell 8: ARCHITECTURE COMPARISON VISUALIZATION

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# A. GPT-J Gain Distribution
ax1 = axes[0, 0]
ax1.hist(df['Last_Gain'], bins=20, alpha=0.7, color='#2ca02c', edgecolor='black')
ax1.axvline(1.0, ls='--', c='red', lw=2, label='Neutral (1.0)')
ax1.axvline(0.80, ls=':', c='blue', lw=2, label='Pythia Ref (0.80)')
ax1.axvline(mean_gain, ls='-', c='green', lw=2, label=f'GPT-J Mean ({mean_gain:.3f})')
ax1.set_xlabel('Last Layer Gain')
ax1.set_ylabel('Count')
ax1.set_title('A. GPT-J-6B Gain Distribution')
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# B. Architecture Comparison Bar Chart
ax2 = axes[0, 1]

# Separate by architecture
parallel_models = [
    ('Pythia-6.9B', 0.80, 'Reference'),
    ('GPT-J-6B', mean_gain, 'This Test')
]
sequential_models = [
    ('GPT2-Medium', 1.16, 'LayerNorm'),
    ('GPT2-XL', 1.02, 'LayerNorm'),
    ('GPT2-Large', 1.01, 'LayerNorm')
]
rmsnorm_models = [
    ('Mistral-7B', 1.11, 'RMSNorm'),
    ('LLaMA-3.1-8B', 1.48, 'RMSNorm'),
    ('Gemma-7B', 2.31, 'RMSNorm')
]

all_models = parallel_models + sequential_models + rmsnorm_models
all_models_sorted = sorted(all_models, key=lambda x: x[1])

names = [m[0] for m in all_models_sorted]
gains = [m[1] for m in all_models_sorted]
colors = []
for m in all_models_sorted:
    if m[0] in ['Pythia-6.9B', 'GPT-J-6B']:
        colors.append('#2ca02c')  # Green for parallel
    elif m[0].startswith('GPT2'):
        colors.append('#1f77b4')  # Blue for sequential LayerNorm
    else:
        colors.append('#ff7f0e')  # Orange for RMSNorm

bars = ax2.barh(range(len(names)), gains, color=colors, alpha=0.8)
ax2.axvline(1.0, ls='--', c='red', lw=2, alpha=0.7)
ax2.set_yticks(range(len(names)))
ax2.set_yticklabels(names)
ax2.set_xlabel('Mean Last Layer Gain')
ax2.set_title('B. Architecture Comparison\n(Green=Parallel, Blue=Sequential, Orange=RMSNorm)')

# Add value labels
for bar, val in zip(bars, gains):
    ax2.text(val + 0.03, bar.get_y() + bar.get_height()/2, 
             f'{val:.2f}', ha='left', va='center', fontsize=10, fontweight='bold')

ax2.grid(True, alpha=0.3, axis='x')
ax2.set_xlim(0, max(gains) * 1.15)

# C. Gain by Category
ax3 = axes[1, 0]
category_order = ['Factual', 'Cliche', 'Nonsense', 'Novel', 'Syntactic']
df_sorted = df.copy()
df_sorted['Category'] = pd.Categorical(df_sorted['Category'], categories=category_order, ordered=True)
df_sorted = df_sorted.sort_values('Category')

sns.boxplot(data=df_sorted, x='Category', y='Last_Gain', ax=ax3, color='#2ca02c')
ax3.axhline(1.0, ls='--', c='red', alpha=0.5, label='Neutral')
ax3.axhline(0.80, ls=':', c='blue', alpha=0.5, label='Pythia Ref')
ax3.set_xlabel('Prompt Category')
ax3.set_ylabel('Last Layer Gain')
ax3.set_title('C. GPT-J-6B Gain by Category')
ax3.tick_params(axis='x', rotation=45)
ax3.legend(loc='upper right')

# D. Entropy vs Gain
ax4 = axes[1, 1]
ax4.scatter(df['Entropy'], df['Last_Gain'], c='#2ca02c', alpha=0.7, s=80, edgecolors='black')
ax4.axhline(1.0, ls='--', c='red', alpha=0.5, label='Neutral (1.0)')
ax4.set_xlabel('Output Entropy (nats)')
ax4.set_ylabel('Last Layer Gain')
ax4.set_title('D. Entropy vs Gain (GPT-J-6B)')

# Add correlation
r, p = pearsonr(df['Entropy'], df['Last_Gain'])
ax4.text(0.05, 0.95, f'r = {r:.3f}\np = {p:.4f}', transform=ax4.transAxes,
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PNG_MAIN, dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFigure saved: {PNG_MAIN}")

In [ ]:
# Cell 9: ARCHITECTURE DICHOTOMY VISUALIZATION

fig, ax = plt.subplots(1, 1, figsize=(14, 8))

# Prepare data organized by architecture
parallel_data = [
    ('Pythia-6.9B', 0.80, 'GPT-NeoX'),
    ('GPT-J-6B', mean_gain, 'GPT-NeoX')
]

sequential_data = [
    ('GPT2-Large', 1.01, 'GPT-2'),
    ('GPT2-XL', 1.02, 'GPT-2'),
    ('GPT2-Medium', 1.16, 'GPT-2')
]

# Create grouped bar chart
width = 0.35
x_parallel = np.arange(len(parallel_data))
x_sequential = np.arange(len(sequential_data)) + len(parallel_data) + 0.5

# Parallel (GPT-NeoX) bars
bars1 = ax.bar(x_parallel, [d[1] for d in parallel_data], width, 
               label='Parallel (GPT-NeoX)', color='#2ca02c', alpha=0.8)

# Sequential (GPT-2) bars
bars2 = ax.bar(x_sequential, [d[1] for d in sequential_data], width,
               label='Sequential (GPT-2)', color='#1f77b4', alpha=0.8)

# Formatting
ax.axhline(1.0, ls='--', c='red', lw=2, label='Neutral (1.0)')
ax.set_ylabel('Mean Last Layer Gain', fontsize=12)
ax.set_title('Parallel vs Sequential Attention Architecture\n(LayerNorm Models Only)', fontsize=14)

# X-axis labels
all_x = list(x_parallel) + list(x_sequential)
all_labels = [d[0] for d in parallel_data] + [d[0] for d in sequential_data]
ax.set_xticks(all_x)
ax.set_xticklabels(all_labels, rotation=45, ha='right')

# Add value labels
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=11, fontweight='bold')

for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=11, fontweight='bold')

# Add annotations
ax.annotate('DAMPENING\n(Gain < 1.0)', xy=(0.15, 0.25), xycoords='axes fraction',
            fontsize=14, ha='center', color='#2ca02c', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
ax.annotate('EXPANSION\n(Gain > 1.0)', xy=(0.75, 0.75), xycoords='axes fraction',
            fontsize=14, ha='center', color='#1f77b4', fontweight='bold',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax.legend(loc='upper left')
ax.grid(True, alpha=0.3, axis='y')
ax.set_ylim(0, 1.4)

plt.tight_layout()
plt.savefig(PNG_ARCH, dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFigure saved: {PNG_ARCH}")

In [ ]:
# Cell 10: SAVE COMPREHENSIVE JSON RESULTS

def make_serializable(obj):
    if isinstance(obj, (np.integer, np.floating)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, dict):
        return {k: make_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [make_serializable(v) for v in obj]
    return obj

# Compile all results
results_json = {
    "experiment": "GPT-J-6B Parallel Architecture Test",
    "purpose": "Validate hypothesis that Parallel Attention (GPT-NeoX) causes dampening",
    "date": TIMESTAMP,
    "model": "GPT-J-6B",
    "architecture": "GPT-NeoX (Parallel Attention)",
    "norm_type": "LayerNorm",
    "n_prompts": total_prompts,
    "n_measurements": len(df),
    
    "results": {
        "mean_gain": float(mean_gain),
        "std_gain": float(std_gain),
        "min_gain": float(min_gain),
        "max_gain": float(max_gain),
        "t_statistic": float(t_stat),
        "p_value_one_sided": float(p_one_sided)
    },
    
    "category_means": make_serializable(df.groupby('Category')['Last_Gain'].mean().to_dict()),
    
    "hypothesis_test": {
        "hypothesis": "Parallel Attention Architecture causes dampening (Gain < 1.0)",
        "result": hypothesis_status,
        "evidence": f"GPT-J-6B shows Gain = {mean_gain:.4f}",
        "comparison": {
            "pythia_6.9b": {"gain": 0.80, "architecture": "Parallel"},
            "gpt2_xl": {"gain": 1.02, "architecture": "Sequential"},
            "gptj_6b": {"gain": float(mean_gain), "architecture": "Parallel"}
        }
    },
    
    "reference_models": make_serializable(REFERENCE_MODELS),
    "all_results": make_serializable(all_results)
}

with open(JSON_FILE, 'w') as f:
    json.dump(results_json, f, indent=2, default=str)

print(f"Results saved to {JSON_FILE}")

In [ ]:
# Cell 11: FINAL VERDICT

print("\n" + "="*70)
print("FINAL VERDICT: PARALLEL ARCHITECTURE HYPOTHESIS")
print("="*70)

print(f"""
┌─────────────────────────────────────────────────────────────────────────────┐
│                    GPT-J-6B PARALLEL ARCHITECTURE TEST                       │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   Model: GPT-J-6B (EleutherAI)                                              │
│   Architecture: GPT-NeoX (Parallel Attention)                               │
│   Normalization: LayerNorm                                                   │
│                                                                              │
│   MEASURED GAIN: {mean_gain:.4f}                                                     │
│   t-test vs 1.0: t = {t_stat:.4f}, p = {p_one_sided:.6f}                              │
│                                                                              │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   REFERENCE COMPARISON:                                                      │
│                                                                              │
│   PARALLEL ATTENTION (GPT-NeoX):                                            │
│     Pythia-6.9B:  0.80  (DAMPENING)                                         │
│     GPT-J-6B:     {mean_gain:.2f}  ({"DAMPENING" if mean_gain < 1.0 else "EXPANSION"})                                         │
│                                                                              │
│   SEQUENTIAL ATTENTION (GPT-2):                                              │
│     GPT2-XL:      1.02  (EXPANSION)                                         │
│     GPT2-Large:   1.01  (EXPANSION)                                         │
│     GPT2-Medium:  1.16  (EXPANSION)                                         │
│                                                                              │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│   HYPOTHESIS: Parallel Attention causes dampening                            │
│   STATUS: {hypothesis_status}                                                      │
│                                                                              │
""")

if hypothesis_status == "CONFIRMED":
    print("│   CONCLUSION:                                                            │")
    print("│   ✅ Both GPT-NeoX models (Pythia, GPT-J) show Gain < 1.0                │")
    print("│   ✅ Both GPT-2 models show Gain > 1.0                                   │")
    print("│   ✅ The PARALLEL ATTENTION architecture causes DAMPENING               │")
    print("│   ✅ This is NOT a Pythia-specific artifact                             │")
    print("│                                                                          │")
    print("│   PAPER #3 CLAIM: VALIDATED                                             │")
else:
    print("│   CONCLUSION:                                                            │")
    print("│   ❌ GPT-J does NOT show the expected dampening                          │")
    print("│   ❌ The parallel architecture hypothesis is REJECTED                    │")
    print("│   ❌ Pythia's dampening remains unexplained                              │")
    print("│                                                                          │")
    print("│   FURTHER INVESTIGATION NEEDED                                          │")

print("│                                                                              │")
print("└─────────────────────────────────────────────────────────────────────────────┘")
print("\n")

In [ ]:
# Cell 12: DOWNLOAD ALL RESULTS

print("\n" + "="*70)
print("DOWNLOADING RESULTS")
print("="*70)

# List all files to download
files_to_download = [CSV_FILE, JSON_FILE, PNG_MAIN, PNG_ARCH]

print("\nFiles to download:")
for f in files_to_download:
    if os.path.exists(f):
        size = os.path.getsize(f) / 1024  # KB
        print(f"  {f} ({size:.1f} KB)")
    else:
        print(f"  {f} (NOT FOUND)")

print("\nStarting downloads...")

# Download each file
for f in files_to_download:
    if os.path.exists(f):
        try:
            files.download(f)
            print(f"  Downloaded: {f}")
        except Exception as e:
            print(f"  FAILED to download {f}: {e}")
    else:
        print(f"  SKIPPED (not found): {f}")

print("\n" + "="*70)
print("ALL DOWNLOADS COMPLETE")
print("="*70)
print("\nNext steps:")
print("  1. If CONFIRMED: Update PYTHIA_ANOMALY_INVESTIGATION.md")
print("  2. If REJECTED: Investigate alternative hypotheses")
print("  3. Add results to Paper #3 documentation")